In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sparrow as sp

In [ ]:
import os

from sparrow.table.pixel_clustering._cluster_intensity import _export_to_ark_format

path="/Users/arnedf/VIB/DATA/ark_analysis"
output_file=os.path.join( path, "average_intensities_SOM_clusters.csv" )

sdata = sp.datasets.cluster_blobs(
        shape=(512, 512), n_cell_types=10, n_cells=100, noise_level_channels=1.2, noise_level_nuclei=1.2, seed=10
    )

img_layer = "blobs_image"
channels = ["lineage_0", "lineage_1", "lineage_5", "lineage_9"]

sdata=sp.im.pixel_clustering_preprocess(
    sdata,
    img_layer=[ img_layer  ],
    output_layer=[ f"{img_layer}_preprocessed" ],
    channels=channels,
    chunks=512,
    overwrite=True,
    sigma=0.0,
)

sdata, fsom, mapping=sp.im.flowsom(
    sdata,
    img_layer=[ f"{img_layer}_preprocessed" ],
    output_layer_clusters=[ f"{img_layer}_flowsom_clusters" ],
    output_layer_metaclusters=[ f"{img_layer}_flowsom_metaclusters" ],
    n_clusters=20,
    random_state=111,
    fraction=0.5,
    chunks=512,
    overwrite=True,
 )

sdata=sp.tb.cluster_intensity(
    sdata,
    mapping=mapping,
    img_layer=f"{img_layer}_preprocessed",
    labels_layer=f"{img_layer}_flowsom_clusters",
    output_layer="counts_clusters",
    overwrite=True,
        )

df=_export_to_ark_format( adata=sdata[ "counts_clusters" ], output=output_file )

In [ ]:
df.head()

Next in an environment where ark is installed run the following cell in a jupyter notebook:

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
from ark.utils.metacluster_remap_gui import MetaClusterGui, metaclusterdata_from_files
from matplotlib import rc_file_defaults

rc_file_defaults()
plt.ion()


pixel_mcd = metaclusterdata_from_files(
    output_file,
    cluster_type='pixel'
)
pixel_mcd.output_mapping_filename = os.path.join(path, "pixel_meta_cluster_mapping.csv" )
pixel_mcg = MetaClusterGui(pixel_mcd, width=17)